In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_file = "../input/global-wheat-detection/train.csv"
train_csv = pd.read_csv(train_file)

In [ ]:
#initial look at the metadata
train_csv.head()

Firstly, we can see that the height and width of all the images shown are the same. Is this the case for the whole data set?

In [ ]:
print(set(train_csv['height']), set(train_csv['width']))

Yes! All the images have the same dimentions. Perhaps this could be manipulated to see how this affects the accuracy of the model.

In [ ]:
# Dimensions of  metadata
train_csv.shape

Tells me that there are 147793 bounding boxes in the training dataset. Will now see how many images don't have any bounding boxes.

In [ ]:
# Number of unique image_id's containing bounding boxes
print(len(set(train_csv['image_id'])))

In [ ]:
3422-3373

There are 3373 images containing bounding boxes. We know that the total number of images was 3422. Therefore, 49 images don't have bounding boxes.

In [ ]:
print(set(train_csv['source']))

The images come from 7 different sources.

# Data Visulisation

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from tqdm import tqdm
import csv

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageDraw, ImageEnhance
import albumentations as albu
from tqdm.notebook import tqdm

In [ ]:
train = "../input/global-wheat-detection/train/"

In [ ]:
#loading the bounding boxes
bboxes = {}
for i, image_id in enumerate(train_csv['image_id']):
    bbox = train_csv['bbox'][i][1:-1]
    bbox = bbox.split(',')
    xmin = float(bbox[0])
    ymin = float(bbox[1])
    width = float(bbox[2])
    height = float(bbox[3])
    
    if image_id not in bboxes:
        bboxes[image_id] = []
        bboxes[image_id].append([xmin, ymin, width, height])
    else:
        bboxes[image_id].append([xmin, ymin, width, height])

In [ ]:
# visulising the images with the bounding boxes
for i, image_id in enumerate(bboxes):
    image_path = train + image_id + '.jpg'
    im = np.array(Image.open(image_path), dtype=np.uint8)
    fig,ax = plt.subplots(1)
    ax.imshow(im)
    
    for xmin, ymin, width, height in bboxes[image_id]:
        rect = patches.Rectangle( (xmin ,ymin), width, height, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
    
    if i == 5:
        break

Here we can see 5 different images showing that the bounding boxes are picking up the wheat heads. There are some annomilies,on the 4th picture, in the top left of the image there is a very large bounding box with several smaller ones within it. On the 5th image, there are some tiny bounding boxes which is difficult to make out with the human eye whether there is a wheat head there or not. 

I will now run a model to see what the initial accuracy of detecting the wheat head is, then I can make small adjustments to try and improve the accuracy.